In [1]:
from sklearn.datasets import load_wine

In [2]:
wine_data = load_wine()

In [3]:
type(wine_data)

sklearn.utils.Bunch

In [4]:
wine_data.keys()

dict_keys(['DESCR', 'data', 'target', 'feature_names', 'target_names'])

In [6]:
print(wine_data['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [7]:
feat_data = wine_data['data']

In [8]:
labels = wine_data['target']

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test =train_test_split(feat_data,labels,test_size=0.3,random_state=101)

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaler = MinMaxScaler()

In [14]:
scaled_X_train = scaler.fit_transform(X_train)

In [15]:
scaled_X_test = scaler.transform(X_test)

In [16]:
import tensorflow as tf

In [18]:
from tensorflow import estimator

In [21]:
X_train.shape


(124, 13)

In [24]:
feat_cols = [tf.feature_column.numeric_column('x',shape=[13])]#shape 13 means no of attributes

In [25]:
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13],feature_columns=feat_cols,
                                    n_classes=3,optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))
# n_classes=3 because we have 3 types of wines

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\HP\\AppData\\Local\\Temp\\tmp47fo5nay', '_session_config': None, '_tf_random_seed': 1, '_save_summary_steps': 100, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000}


In [27]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_X_train},
                                          y=y_train,
                                          shuffle=True,
                                          batch_size=10,
                                          num_epochs=5)
#input_fn is kind of feed dictionary

In [28]:
deep_model.train(input_fn=input_fn,steps=500)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\HP\AppData\Local\Temp\tmp47fo5nay\model.ckpt.
INFO:tensorflow:step = 1, loss = 10.266
INFO:tensorflow:Saving checkpoints for 62 into C:\Users\HP\AppData\Local\Temp\tmp47fo5nay\model.ckpt.
INFO:tensorflow:Loss for final step: 4.65663.


In [29]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_X_test},
                                               shuffle=False)

In [30]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Restoring parameters from C:\Users\HP\AppData\Local\Temp\tmp47fo5nay\model.ckpt-62


In [31]:
preds

[{'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object),
  'logits': array([-0.29548085,  0.90145725, -1.74400985], dtype=float32),
  'probabilities': array([ 0.22002773,  0.72828436,  0.05168794], dtype=float32)},
 {'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object),
  'logits': array([-0.39533404,  1.09323442, -2.18879485], dtype=float32),
  'probabilities': array([ 0.17866296,  0.79161054,  0.02972654], dtype=float32)},
 {'class_ids': array([2], dtype=int64),
  'classes': array([b'2'], dtype=object),
  'logits': array([-2.59471846, -0.12201887,  2.34365654], dtype=float32),
  'probabilities': array([ 0.00656178,  0.07778599,  0.91565228], dtype=float32)},
 {'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object),
  'logits': array([ 1.39332819,  0.10535878, -2.51762462], dtype=float32),
  'probabilities': array([ 0.77169323,  0.2128564 ,  0.01545039], dtype=float32)},
 {'class_ids': array([2], dtype=int64),


In [32]:
predictions = [p['class_ids'][0] for p in preds]# to see better view we take the first postion {'class_ids': array([1], dtype=int64) of this [0] 

In [33]:
predictions

[1,
 1,
 2,
 0,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 0,
 1,
 1,
 0]

In [35]:
from sklearn.metrics import confusion_matrix, classification_report    

In [36]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       1.00      0.42      0.59        19
          1       0.64      0.95      0.76        22
          2       0.92      0.92      0.92        13

avg / total       0.83      0.76      0.74        54

